In [32]:
import matplotlib.pylab as plt
import pandas as pd
import sklearn.metrics
import sklearn.neighbors
iris = pd.read_csv('iris_dataset.csv')

iris.columns = ["sepal_length", "sepal_width", "petal_length", "petal_width", "class"]
#iris['Number'] = iris.index + 1
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,class
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa


In [31]:
train_data, test_data = sklearn.model_selection.train_test_split(iris,test_size=0.4, random_state=26)
train_data.head()

,sepal_length,sepal_width,petal_length,petal_width,class,Number
106,7.3,2.9,6.3,1.8,Iris-virginica,107
42,5.0,3.5,1.6,0.6,Iris-setosa,43
141,5.8,2.7,5.1,1.9,Iris-virginica,142
9,5.4,3.7,1.5,0.2,Iris-setosa,10
84,6.0,3.4,4.5,1.6,Iris-versicolor,85


# KNN model

In [33]:
predict_data = pd.DataFrame([{'sepal_length': 5, 'sepal_width': 3, 'petal_length': 2, 'petal_width': 0.5},])
print(predict_data)
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(train_data[["sepal_length", "sepal_width", "petal_length", "petal_width"]])

   sepal_length  sepal_width  petal_length  petal_width
0             5            3             2          0.5


StandardScaler()

In [34]:
scaler = sklearn.preprocessing.StandardScaler()
scaler.fit(train_data[["sepal_length", "sepal_width", "petal_length", "petal_width"]])

iris_normalized = pd.concat([pd.DataFrame(scaler.transform(
        iris[["sepal_length", "sepal_width", "petal_length", "petal_width"]]), columns=["sepal_length", "sepal_width", "petal_length", "petal_width"]),
        iris["class"]], axis=1)
train_normalized = iris_normalized.iloc[train_data.index]
test_normalized = iris_normalized.iloc[test_data.index]
predict_normalized = pd.DataFrame(scaler.transform(predict_data),
                                      columns=["sepal_length", "sepal_width", "petal_length", "petal_width"])

In [36]:
train_normalized_x = train_normalized[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
train_normalized_y = train_normalized["class"]
test_normalized_x = test_normalized[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
test_normalized_y = test_normalized["class"]

In [40]:
results = []
for k in range(1, 15):
    knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors=k).fit(train_normalized_x, train_normalized_y)
    results.append({'k': k, 'accuracy': sklearn.metrics.accuracy_score(test_normalized_y, knn.predict(test_normalized_x))})
results = pd.DataFrame(results)
print(results)

# train with all data
iris_x = iris_normalized[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
iris_y = iris_normalized["class"]
knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors=4).fit(iris_x, iris_y)
distances, indices = knn.kneighbors(predict_normalized)

predictions = knn.predict(predict_normalized)
for index, prediction in enumerate(predictions):
    print(64 * '-')
    print(prediction)
    print(predict_data.iloc[index])
    print(iris.iloc[indices[index], :])
    print(iris_normalized.iloc[indices[index], :])

     k  accuracy
0    1  0.900000
1    2  0.916667
2    3  0.933333
3    4  0.950000
4    5  0.950000
5    6  0.966667
6    7  0.950000
7    8  0.950000
8    9  0.950000
9   10  0.966667
10  11  0.966667
11  12  0.966667
12  13  0.950000
13  14  0.966667
----------------------------------------------------------------
Iris-setosa
sepal_length    5.0
sepal_width     3.0
petal_length    2.0
petal_width     0.5
Name: 0, dtype: float64
    sepal_length  sepal_width  petal_length  petal_width        class
24           5.0          3.0           1.6          0.2  Iris-setosa
44           4.8          3.0           1.4          0.3  Iris-setosa
0            4.9          3.0           1.4          0.2  Iris-setosa
33           4.9          3.1           1.5          0.2  Iris-setosa
    sepal_length  sepal_width  petal_length  petal_width        class
24     -1.055896    -0.013555     -1.261568    -1.364277  Iris-setosa
44     -1.310915    -0.013555     -1.378222    -1.227696  Iris-setosa
0   

# Logestic Model

In [73]:
pip install mord

  Created wheel for mord: filename=mord-0.6-py3-none-any.whl size=6006 sha256=1d542accbad0d1066134d15b87b3a879ac134a791e4a8d06b22cf8b55b624a89
  Stored in directory: c:\users\chenyushang\appdata\local\pip\cache\wheels\1d\94\48\b3d11f5f91d26bec0c95c6eb9c27c6e427d0b77fb2406d0e0a
Successfully built mord
Note: you may need to restart the kernel to use updated packages.


In [74]:
%matplotlib inline
import matplotlib.pylab as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.api as sm

from dmba import classificationSummary, gainsChart, liftChart
from dmba.metric import AIC_score
from mord import LogisticIT
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split

In [85]:
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from dmba.metric import AIC_score


iris2 = iris.copy()
encode = defaultdict(LabelEncoder)
encoded = iris2.apply(lambda x: encode[x.name].fit_transform(x))
encoded = encoded.drop(['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], axis=1)

encoded['sepal_length'] = iris2['sepal_length']
encoded['sepal_width'] = iris2['sepal_width']
encoded['petal_length'] = iris2['petal_length']
encoded['petal_width'] = iris2['petal_width']

X = encoded.drop(['class'], axis=1)
y = encoded["class"]

train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

# fit a logistic regression (set penalty=l2 and C=1e42 to avoid regularization)
logit_reg = LogisticRegression(penalty="l2", C=1e42, solver='liblinear')
logit_reg.fit(train_X, train_y)

print('intercept ', logit_reg.intercept_[0])
print(pd.DataFrame({'coeff': logit_reg.coef_[0]}, index=X.columns).transpose())
print()
# Akaike information criterion (AIC) (Akaike, 1974) is a fined technique based on in-sample 
# fit to estimate the likelihood of a model to predict/estimate the future values.
print('AIC', AIC_score(valid_y, logit_reg.predict(valid_X), df = len(train_X.columns) + 1))

     sepal_length  sepal_width  petal_length  petal_width
0             4.9          3.0           1.4          0.2
1             4.7          3.2           1.3          0.2
2             4.6          3.1           1.5          0.2
3             5.0          3.6           1.4          0.2
4             5.4          3.9           1.7          0.4
..            ...          ...           ...          ...
144           6.7          3.0           5.2          2.3
145           6.3          2.5           5.0          1.9
146           6.5          3.0           5.2          2.0
147           6.2          3.4           5.4          2.3
148           5.9          3.0           5.1          1.8

[149 rows x 4 columns]
0      0
1      0
2      0
3      0
4      0
      ..
144    2
145    2
146    2
147    2
148    2
Name: class, Length: 149, dtype: int32
intercept  0.7823644689570716
       sepal_length  sepal_width  petal_length  petal_width
coeff      1.082571      5.39968     -7.560075    -3

# Bayes Model

In [97]:
# import required packages
%matplotlib inline

import matplotlib.pylab as plt
import pandas as pd
from dmba import classificationSummary, gainsChart
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
iris3 = iris.copy()

In [101]:
iris3.head(50)

,sepal_length,sepal_width,petal_length,petal_width,class
0,4.9,3.0,1.4,0.2,Iris-setosa
1,4.7,3.2,1.3,0.2,Iris-setosa
2,4.6,3.1,1.5,0.2,Iris-setosa
3,5.0,3.6,1.4,0.2,Iris-setosa
4,5.4,3.9,1.7,0.4,Iris-setosa
5,4.6,3.4,1.4,0.3,Iris-setosa
6,5.0,3.4,1.5,0.2,Iris-setosa
7,4.4,2.9,1.4,0.2,Iris-setosa
8,4.9,3.1,1.5,0.1,Iris-setosa
9,5.4,3.7,1.5,0.2,Iris-setosa


In [135]:
iris3 = iris3.astype('category')
predictors = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
X = pd.get_dummies(iris3[predictors])
y = (iris3['class'] == 'Iris-versicolor').astype(int)
classes = ['Iris-versicolor', 'Other']

In [136]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=1)

In [137]:
delays_nb = MultinomialNB(alpha=0.01)
delays_nb.fit(X_train, y_train)

MultinomialNB(alpha=0.01)

In [138]:
predProb_train = delays_nb.predict_proba(X_train)
predProb_valid = delays_nb.predict_proba(X_valid)

In [139]:
y_valid_pred = delays_nb.predict(X_valid)
y_train_pred = delays_nb.predict(X_train)

In [140]:
print('Training\n')
classificationSummary(y_train, y_train_pred, class_names=classes) 
print('\n\nTesting\n')
classificationSummary(y_valid, y_valid_pred, class_names=classes)

Training

Confusion Matrix (Accuracy 0.9888)

                Prediction
         Actual Iris-versicolor           Other
Iris-versicolor              58               1
          Other               0              30


Testing

Confusion Matrix (Accuracy 0.9000)

                Prediction
         Actual Iris-versicolor           Other
Iris-versicolor              37               3
          Other               3              17


In [144]:
X = pd.get_dummies(iris3[predictors])
y = (iris3['class'] == 'Iris-setosa').astype(int)
classes = ['Iris-setosa', 'Other']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=1)
delays_nb = MultinomialNB(alpha=0.01)
delays_nb.fit(X_train, y_train)
predProb_train = delays_nb.predict_proba(X_train)
predProb_valid = delays_nb.predict_proba(X_valid)
y_valid_pred = delays_nb.predict(X_valid)
y_train_pred = delays_nb.predict(X_train)
print('Training\n')
classificationSummary(y_train, y_train_pred, class_names=classes) 
print('\n\nTesting\n')
classificationSummary(y_valid, y_valid_pred, class_names=classes)

Training

Confusion Matrix (Accuracy 1.0000)

            Prediction
     Actual Iris-setosa       Other
Iris-setosa          60           0
      Other           0          29


Testing

Confusion Matrix (Accuracy 0.9833)

            Prediction
     Actual Iris-setosa       Other
Iris-setosa          39           1
      Other           0          20


In [145]:
X = pd.get_dummies(iris3[predictors])
y = (iris3['class'] == 'Iris-virginica').astype(int)
classes = ['Iris-virginica', 'Other']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.40, random_state=1)
delays_nb = MultinomialNB(alpha=0.01)
delays_nb.fit(X_train, y_train)
predProb_train = delays_nb.predict_proba(X_train)
predProb_valid = delays_nb.predict_proba(X_valid)
y_valid_pred = delays_nb.predict(X_valid)
y_train_pred = delays_nb.predict(X_train)
print('Training\n')
classificationSummary(y_train, y_train_pred, class_names=classes) 
print('\n\nTesting\n')
classificationSummary(y_valid, y_valid_pred, class_names=classes)

Training

Confusion Matrix (Accuracy 0.9775)

               Prediction
        Actual Iris-virginica          Other
Iris-virginica             57              2
         Other              0             30


Testing

Confusion Matrix (Accuracy 0.9000)

               Prediction
        Actual Iris-virginica          Other
Iris-virginica             36              4
         Other              2             18


# Comment
* Based on the result above I think that the Bayes model is the best with the highest accuracy. Knn model is the second and logistic model is the worst. Since logistic model is not use to predict y which is categorical variable. 

# Excercise 2

In [147]:
day = pd.read_csv('day.csv')
day['Number'] = day.index + 1
day.head()


,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,Number
0,1,2011-01-01,1,0,1,0,6,0,2,0.344167,0.363625,0.805833,0.160446,331,654,985,1
1,2,2011-01-02,1,0,1,0,0,0,2,0.363478,0.353739,0.696087,0.248539,131,670,801,2
2,3,2011-01-03,1,0,1,0,1,1,1,0.196364,0.189405,0.437273,0.248309,120,1229,1349,3
3,4,2011-01-04,1,0,1,0,2,1,1,0.200000,0.212122,0.590435,0.160296,108,1454,1562,4
4,5,2011-01-05,1,0,1,0,3,1,1,0.226957,0.229270,0.436957,0.186900,82,1518,1600,5


In [148]:
hour = pd.read_csv('hour.csv')
hour['Number'] = hour.index + 1
hour.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt,Number
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16,1
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40,2
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32,3
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13,4
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1,5


In [154]:
%matplotlib inline
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, BayesianRidge
import statsmodels.formula.api as sm
import matplotlib.pylab as plt

from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score

In [157]:
predictors = ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed']
outcome = 'casual'
X = pd.get_dummies(hour[predictors], drop_first=True)
y = hour[outcome]
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=9)
hr_lm = LinearRegression()
hr_lm.fit(train_X, train_y)
print('intercept ', hr_lm.intercept_)
print(pd.DataFrame({'Predictor': X.columns, 'coefficient': hr_lm.coef_}))

# print performance measures
regressionSummary(train_y, hr_lm.predict(train_X))

intercept  18.62572634831102
     Predictor  coefficient
0       season     1.928786
1           yr    10.577704
2         mnth    -0.111494
3           hr     1.155775
4      holiday   -14.937562
5      weekday     0.918831
6   workingday   -36.254171
7   weathersit     2.258657
8         temp    44.275145
9        atemp    71.202445
10         hum   -69.284493
11   windspeed     6.309983

Regression statistics

               Mean Error (ME) : -0.0000
Root Mean Squared Error (RMSE) : 36.5316
     Mean Absolute Error (MAE) : 24.5905


In [160]:
pred_y = hr_lm.predict(train_X)

print('adjusted r2 : ', adjusted_r2_score(train_y, pred_y, hr_lm))
print('AIC : ', AIC_score(train_y, pred_y, hr_lm))
print('BIC : ', BIC_score(train_y, pred_y, hr_lm))

adjusted r2 :  0.45128755401478093
AIC :  104654.93679390431
BIC :  104756.46694815137


In [162]:
# Use predict() to make predictions on a new set
hr_lm_pred = hr_lm.predict(valid_X)

result = pd.DataFrame({'Predicted': hr_lm_pred, 'Actual': valid_y,
                       'Residual': valid_y - hr_lm_pred})
print(result.head(20))

# Compute common accuracy measures
regressionSummary(valid_y, hr_lm_pred)

       Predicted  Actual    Residual
4541   38.208496       0  -38.208496
6457   53.990010      41  -12.990010
8653   13.835905       4   -9.835905
6104   28.274945      32    3.725055
12203  94.914761     223  128.085239
8518   37.616914      28   -9.616914
10376  23.157313      64   40.842687
16526  -2.048044       9   11.048044
2421   33.811772      41    7.188228
13516  74.774873      79    4.225127
15891   7.905359       5   -2.905359
10390  16.254931       2  -14.254931
257   -50.235996       0   50.235996
6653   61.984740     116   54.015260
14602  62.361677      41  -21.361677
14434  75.900713      96   20.099287
14875  15.424384      14   -1.424384
11481  79.698438     147   67.301562
8200  -20.016272       9   29.016272
17293   5.781874      77   71.218126

Regression statistics

               Mean Error (ME) : -0.3194
Root Mean Squared Error (RMSE) : 36.1545
     Mean Absolute Error (MAE) : 24.5568


In [163]:
predictors = ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed']
outcome = 'registered'
X = pd.get_dummies(hour[predictors], drop_first=True)
y = hour[outcome]
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=9)
hr_lm = LinearRegression()
hr_lm.fit(train_X, train_y)
print('intercept ', hr_lm.intercept_)
print(pd.DataFrame({'Predictor': X.columns, 'coefficient': hr_lm.coef_}))

# print performance measures
regressionSummary(train_y, hr_lm.predict(train_X))

pred_y = hr_lm.predict(train_X)

print('adjusted r2 : ', adjusted_r2_score(train_y, pred_y, hr_lm))
print('AIC : ', AIC_score(train_y, pred_y, hr_lm))
print('BIC : ', BIC_score(train_y, pred_y, hr_lm))

# Use predict() to make predictions on a new set
hr_lm_pred = hr_lm.predict(valid_X)

result = pd.DataFrame({'Predicted': hr_lm_pred, 'Actual': valid_y,
                       'Residual': valid_y - hr_lm_pred})
print(result.head(20))

# Compute common accuracy measures
regressionSummary(valid_y, hr_lm_pred)

intercept  -47.72167960951376
     Predictor  coefficient
0       season    19.105118
1           yr    73.822322
2         mnth    -0.051911
3           hr     6.381137
4      holiday   -17.093923
5      weekday     1.200914
6   workingday    39.353959
7   weathersit    -5.343654
8         temp    61.339052
9        atemp   133.106449
10         hum  -127.072928
11   windspeed    44.018822

Regression statistics

               Mean Error (ME) : -0.0000
Root Mean Squared Error (RMSE) : 125.4076
     Mean Absolute Error (MAE) : 90.6170
adjusted r2 :  0.33371671451918883
AIC :  130376.09203764357
BIC :  130477.62219189062
        Predicted  Actual    Residual
4541   136.966989       4 -132.966989
6457   124.903042     154   29.096958
8653    37.152097      10  -27.152097
6104   200.294447     135  -65.294447
12203  243.380397     278   34.619603
8518    75.910865      69   -6.910865
10376  173.074705     155  -18.074705
16526  158.655733     629  470.344267
2421    96.553227     168   7

In [164]:
predictors = ['season', 'yr', 'mnth', 'hr', 'holiday', 'weekday', 'workingday', 'weathersit', 'temp', 'atemp', 'hum', 'windspeed']
outcome = 'cnt'
X = pd.get_dummies(hour[predictors], drop_first=True)
y = hour[outcome]
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=9)
hr_lm = LinearRegression()
hr_lm.fit(train_X, train_y)
print('intercept ', hr_lm.intercept_)
print(pd.DataFrame({'Predictor': X.columns, 'coefficient': hr_lm.coef_}))

# print performance measures
regressionSummary(train_y, hr_lm.predict(train_X))

pred_y = hr_lm.predict(train_X)

print('adjusted r2 : ', adjusted_r2_score(train_y, pred_y, hr_lm))
print('AIC : ', AIC_score(train_y, pred_y, hr_lm))
print('BIC : ', BIC_score(train_y, pred_y, hr_lm))

# Use predict() to make predictions on a new set
hr_lm_pred = hr_lm.predict(valid_X)

result = pd.DataFrame({'Predicted': hr_lm_pred, 'Actual': valid_y,
                       'Residual': valid_y - hr_lm_pred})
print(result.head(20))

# Compute common accuracy measures
regressionSummary(valid_y, hr_lm_pred)

intercept  -29.09595326120433
     Predictor  coefficient
0       season    21.033904
1           yr    84.400026
2         mnth    -0.163404
3           hr     7.536912
4      holiday   -32.031485
5      weekday     2.119745
6   workingday     3.099789
7   weathersit    -3.084996
8         temp   105.614197
9        atemp   204.308895
10         hum  -196.357421
11   windspeed    50.328805

Regression statistics

                      Mean Error (ME) : 0.0000
       Root Mean Squared Error (RMSE) : 143.6847
            Mean Absolute Error (MAE) : 107.3616
          Mean Percentage Error (MPE) : -228.7092
Mean Absolute Percentage Error (MAPE) : 335.3326
adjusted r2 :  0.38583992864762895
AIC :  133213.31205684747
BIC :  133314.8422110945
        Predicted  Actual    Residual
4541   175.175485       4 -171.175485
6457   178.893052     195   16.106948
8653    50.988002      14  -36.988002
6104   228.569392     167  -61.569392
12203  338.295158     501  162.704842
8518   113.527780      9